In [104]:
class Operation():
    def __init__(self,input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []

        for node in input_nodes:
            node.output_nodes.append(self)
        
        _default_graph.operations.append(self)
   
    def compute():
        pass

In [105]:
class add(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])

    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var+y_var

In [106]:
class multiply(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])

    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var*y_var

In [107]:
class matmultiply(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])

    def compute(self,x_var,y_var):
        self.inputs = [x_var,y_var]
        return x_var.dot(y_var)

In [108]:
class Placeholder():
    def __init__(self):
        self.output_nodes=[]
        _default_graph.placeholders.append(self)
        

In [109]:
class Variable():
    def __init__(self,initial_value=None):
        self.value = initial_value
        self.output_nodes=[]
        _default_graph.variables.append(self)

In [110]:
class Graph():
    def __init__(self):   
        self.operations = []
        self.placeholders = []
        self.variables =[]
    
    def set_as_default(self):
        global _default_graph
        _default_graph=self

In [111]:
g = Graph()

In [112]:
g.set_as_default()

In [113]:
A = Variable(10)

In [114]:
b = Variable(1)

In [115]:
x = Placeholder()

In [116]:
y = multiply(A,x)

In [117]:
z = add(y,b)

In [118]:
def traverse_postorder(operation):

#         PostOrder Traversal of Nodes. Basically makes sure computations are done in \n",
#         the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.\n",
#         It is not super important for understanding the basic fundamentals of deep learning.\n",

    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [129]:
import numpy as np
class Session():
    def run(self,operation,feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                node.output = node.value
            
            else:
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
            
            if type(node.output)==list:
                node.output = np.array(node.output)
    
        return operation.output      

In [130]:
sess = Session()

In [131]:
result = sess.run(z,feed_dict={x:10})
result

array([[101, 201],
       [111, 221]])

In [138]:
g = Graph()
g.set_as_default()
A = Variable([[110,20],[11,22]])
b = Variable([1,1])
x = Placeholder()
y = matmultiply(A,x)
z = add(y,b)

In [139]:
sess = Session()

In [140]:
sess.run(z,{x:10})

array([[1101,  201],
       [ 111,  221]])